In [1]:
import pandas as pd
import glob
import os
import numpy as np
import pickle as pkl
import time
import dask.dataframe as dd
from fastparquet import write
import fastparquet

In [4]:
all_pkl_files=[]
pkl_file_sizes=[]
path='/data/dharp/compounds/datasets/google/'
for filename in glob.glob(path+'*pkl'):
    pkl_file_sizes.append(os.path.getsize(filename))
    all_pkl_files.append(filename)
#df = pd.concat(dfs)

In [13]:
pkl_df=pd.DataFrame(all_pkl_files,pkl_file_sizes)
pkl_df.reset_index(inplace=True)
pkl_df.columns=['fsize','fname']
pkl_df['fsize_perc']=pkl_df.fsize/pkl_df.fsize.sum()*100
pkl_df.sort_values(by=['fsize'],ascending=False,inplace=True,ignore_index=True)
pkl_df.fsize/=1024*1024
pkl_df

,fsize,fname,fsize_perc
0,25402.093315,/data/dharp/compounds/datasets/google/th2.pkl,2.446683e+00
1,25396.113003,/data/dharp/compounds/datasets/google/th4.pkl,2.446107e+00
2,25395.894462,/data/dharp/compounds/datasets/google/th6.pkl,2.446086e+00
3,25390.465415,/data/dharp/compounds/datasets/google/th1.pkl,2.445563e+00
4,25386.227156,/data/dharp/compounds/datasets/google/th5.pkl,2.445155e+00
...,...,...,...
773,0.001607,/data/dharp/compounds/datasets/google/qg.pkl,1.547776e-07
774,0.001570,/data/dharp/compounds/datasets/google/xw.pkl,1.511952e-07
775,0.001486,/data/dharp/compounds/datasets/google/qy.pkl,1.431119e-07
776,0.001257,/data/dharp/compounds/datasets/google/jq.pkl,1.210664e-07


In [14]:
maxvalue = 30_000

lastvalue = 0
newcum = []
labels=[]
cur_label=1
for row in pkl_df.itertuples():
    thisvalue =  row.fsize + lastvalue
    if thisvalue > maxvalue:
        thisvalue = 0
        cur_label+=1
    newcum.append( thisvalue )
    labels.append(cur_label)
    lastvalue = thisvalue
pkl_df['fcat']=labels
pkl_df

,fsize,fname,fsize_perc,fcat
0,25402.093315,/data/dharp/compounds/datasets/google/th2.pkl,2.446683e+00,1
1,25396.113003,/data/dharp/compounds/datasets/google/th4.pkl,2.446107e+00,2
2,25395.894462,/data/dharp/compounds/datasets/google/th6.pkl,2.446086e+00,2
3,25390.465415,/data/dharp/compounds/datasets/google/th1.pkl,2.445563e+00,3
4,25386.227156,/data/dharp/compounds/datasets/google/th5.pkl,2.445155e+00,3
...,...,...,...,...
773,0.001607,/data/dharp/compounds/datasets/google/qg.pkl,1.547776e-07,29
774,0.001570,/data/dharp/compounds/datasets/google/xw.pkl,1.511952e-07,29
775,0.001486,/data/dharp/compounds/datasets/google/qy.pkl,1.431119e-07,29
776,0.001257,/data/dharp/compounds/datasets/google/jq.pkl,1.210664e-07,29


In [36]:
context_file=pd.read_pickle('/data/dharp/compounds/Compounding/data/contexts.pkl')
contexts=[context.split('_')[0] for context in context_file]
contexts=list(set(contexts))
len(contexts)

40215

In [144]:
cur_parq=fastparquet.ParquetFile('/data/dharp/compounds/datasets/entire_df/df_1.parq')

In [145]:
first = next(iter(cur_parq.iter_row_groups()))

In [146]:
first

,lemma_sent,year,pos_sent,comp_class,ner_sent,count
0,-pron- flying ? end,1987,DET SPACE NOUN PUNCT NOUN,0,,1
1,-pron- flying ? end,1989,DET SPACE NOUN PUNCT NOUN,0,,1
2,-pron- flying ? end,1990,DET SPACE NOUN PUNCT NOUN,0,,22
3,-pron- flying ? end,1992,DET SPACE NOUN PUNCT NOUN,0,,1
4,-pron- flying ? end,1993,DET SPACE NOUN PUNCT NOUN,0,,1
...,...,...,...,...,...,...
47900564,that -pron- have none --,1886,SCONJ PRON AUX NOUN PUNCT,0,,2
47900565,that -pron- have none --,1896,SCONJ PRON AUX NOUN PUNCT,0,,9
47900566,that -pron- have none --,1900,SCONJ PRON AUX NOUN PUNCT,0,,1
47900567,that -pron- have none --,1903,SCONJ PRON AUX NOUN PUNCT,0,,3


In [132]:
def syntactic_reducer(df):
    pattern=df.iloc[0].comp_class
    if pattern==1: # N N _ _ N N
        compound_left_df,modifier_left_df,head_left_df=left_side_parser(df)
        compound_right_df,modifier_right_df,head_right_df=right_side_parser(df)
        
        final_compound_df=pd.concat([compound_left_df,compound_right_df],ignore_index=True)
        final_modifier_df=pd.concat([modifier_left_df,modifier_right_df],ignore_index=True)
        final_head_df=pd.concat([head_left_df,head_right_df],ignore_index=True)
           
    elif pattern==2: # N N _ _ _
        final_compound_df,final_modifier_df,final_head_df=left_side_parser(df)

    elif pattern==3: # _ N N _ _
        final_compound_df,final_modifier_df,final_head_df=mid1_parser(df)
    
    elif pattern==4: # _ _ N N _
        final_compound_df,final_modifier_df,final_head_df=mid2_parser(df)
        
    elif pattern==5: # _ _ _ N N
        final_compound_df,final_modifier_df,final_head_df=right_side_parser(df)

    return final_compound_df,final_modifier_df,final_head_df

In [133]:
def compound_extracter(df):
    if df.loc[df.comp_class==1].shape[0]!=0:
        sides_comp_df,sides_mod_df,sides_head_df=syntactic_reducer(df.loc[df.comp_class==1])
    
    if df.loc[df.comp_class==2].shape[0]!=0:
        left_comp_df,left_mod_df,left_head_df=syntactic_reducer(df.loc[df.comp_class==2])
        
    if df.loc[df.comp_class==3].shape[0]!=0:
        mid1_comp_df,mid1_mod_df,mid1_head_df=syntactic_reducer(df.loc[df.comp_class==3])
        
    if df.loc[df.comp_class==4].shape[0]!=0:
        mid2_comp_df,mid2_mod_df,mid2_head_df=syntactic_reducer(df.loc[df.comp_class==4])

    if df.loc[df.comp_class==4].shape[0]!=0:
        right_comp_df,right_mod_df,right_head_df=syntactic_reducer(df.loc[df.comp_class==5])
        

    compounds=pd.concat([sides_comp_df,left_comp_df,mid1_comp_df,mid2_comp_df,right_comp_df],ignore_index=True,sort=False)
    modifiers=pd.concat([sides_mod_df,left_mod_df,mid1_mod_df,mid2_mod_df,right_mod_df],ignore_index=True,sort=False)
    heads=pd.concat([sides_head_df,left_head_df,mid1_head_df,mid2_head_df,right_head_df],ignore_index=True,sort=False)
    
    
    compounds.dropna(inplace=True)
    compounds=compounds.groupby(['modifier','head','context','year'])['count'].sum().to_frame()
    compounds.reset_index(inplace=True)
    
    modifiers.dropna(inplace=True)
    modifiers=modifiers.groupby(['modifier','context','year'])['count'].sum().to_frame()
    modifiers.reset_index(inplace=True)
    
    heads.dropna(inplace=True)
    heads=heads.groupby(['head','context','year'])['count'].sum().to_frame()
    heads.reset_index(inplace=True)
    
    return compounds,modifiers,heads

In [97]:
compounds=first.loc[first.comp_class!=0].reset_index(drop=True)
b1=compounds.loc[compounds.comp_class==1]
b2=compounds.loc[compounds.comp_class==2]
b3=compounds.loc[compounds.comp_class==3]
b4=compounds.loc[compounds.comp_class==4]
b5=compounds.loc[compounds.comp_class==5]

In [134]:
results=compound_extracter(compounds)

In [137]:
results[2]

,head,context,year,count
0,%,all,1969,2
1,%,all,1971,1
2,%,all,1972,2
3,%,all,1974,5
4,%,all,1976,1
...,...,...,...,...
2417782,°,than,2004,1
2417783,°,than,2005,6
2417784,°,than,2006,6
2417785,°,than,2007,10


In [118]:
def left_side_parser(df): # N N _ _ _
    cur_df=df.copy()
    cur_df[['modifier','head','w1','w2','w3']]=cur_df.lemma_sent.str.split(' ',expand=True)
    compound_df=pd.melt(cur_df,id_vars=['modifier','head','year','count'],value_vars=['w1','w2','w3'],value_name='context')
    compound_df=compound_df.loc[compound_df.context.isin(contexts)]

    modifier_df=pd.melt(cur_df,id_vars=['modifier','year','count'],value_vars=['head','w1','w2'],value_name='context')
    modifier_df=modifier_df.loc[modifier_df.context.isin(contexts)]
    
    head_df=pd.melt(cur_df,id_vars=['head','year','count'],value_vars=['modifier','w1','w2','w3'],value_name='context')
    head_df=head_df.loc[head_df.context.isin(contexts)]
    
    return compound_df,modifier_df,head_df

def mid1_parser(df): # _ N N _ _
    cur_df=df.copy()
    cur_df[['w1','modifier','head','w2','w3']]=cur_df.lemma_sent.str.split(' ',expand=True)
    compound_df=pd.melt(cur_df,id_vars=['modifier','head','year','count'],value_vars=['w1','w2','w3'],value_name='context')
    compound_df=compound_df.loc[compound_df.context.isin(contexts)]

    modifier_df=pd.melt(cur_df,id_vars=['modifier','year','count'],value_vars=['head','w1','w2','w3'],value_name='context')
    modifier_df=modifier_df.loc[modifier_df.context.isin(contexts)]
    
    head_df=pd.melt(cur_df,id_vars=['head','year','count'],value_vars=['modifier','w1','w2','w3'],value_name='context')
    head_df=head_df.loc[head_df.context.isin(contexts)]
    
    return compound_df,modifier_df,head_df

def mid2_parser(df): # _ _ N N _
    cur_df=df.copy()
    cur_df[['w1','w2','modifier','head','w3']]=cur_df.lemma_sent.str.split(' ',expand=True)
    compound_df=pd.melt(cur_df,id_vars=['modifier','head','year','count'],value_vars=['w1','w2','w3'],value_name='context')
    compound_df=compound_df.loc[compound_df.context.isin(contexts)]

    modifier_df=pd.melt(cur_df,id_vars=['modifier','year','count'],value_vars=['head','w1','w2','w3'],value_name='context')
    modifier_df=modifier_df.loc[modifier_df.context.isin(contexts)]
    
    head_df=pd.melt(cur_df,id_vars=['head','year','count'],value_vars=['modifier','w1','w2','w3'],value_name='context')
    head_df=head_df.loc[head_df.context.isin(contexts)]
    
    return compound_df,modifier_df,head_df

def right_side_parser(df): # _ _ _ N N
    cur_df=df.copy()
    cur_df[['w1','w2','w3','modifier','head']]=cur_df.lemma_sent.str.split(' ',expand=True)
    compound_df=pd.melt(cur_df,id_vars=['modifier','head','year','count'],value_vars=['w1','w2','w3'],value_name='context')
    compound_df=compound_df.loc[compound_df.context.isin(contexts)]
    
    modifier_df=pd.melt(cur_df,id_vars=['modifier','year','count'],value_vars=['head','w1','w2','w3'],value_name='context')
    modifier_df=modifier_df.loc[modifier_df.context.isin(contexts)]
    
    head_df=pd.melt(cur_df,id_vars=['head','year','count'],value_vars=['modifier','w2','w3'],value_name='context')
    head_df=head_df.loc[head_df.context.isin(contexts)]
    
    return compound_df,modifier_df,head_df

def write_to_hdf(data_bin):
    if data_bin.shape[0]!=0:
        df_list=[]
        print(data_bin.shape[0])
        for row in data_bin.itertuples():
            #print(row.fname)
            tmp_pkl=pd.read_pickle(row.fname)
            tmp_pkl.reset_index(inplace=True)
            df_list.append(tmp_pkl)
        tmp_df=pd.concat(df_list,ignore_index=True,sort=False)
        print('Done concatenating')
        tmp_df.groupby(['fivegram_pos','year'])['count'].sum().to_frame().reset_index(inplace=True)
        print('Done grouping')
        tmp_df.to_hdf(path+'/entire_df_'+str(row.fcat)+'.h5',format='table', key='df',complib='zlib', complevel=5)

In [33]:
def write_to_parquet(data_bin):
    print(data_bin.iloc[0].fcat)
    return
    cur_time=time.time()
    df_list=[]
    print(data_bin.shape[0])
    for row in data_bin.itertuples():
        #print(row.fname)
        cur_df=pd.read_pickle(row.fname)
        cur_df.reset_index(inplace=True)
        df_list.append(cur_df)
    concat_df=pd.concat(df_list,ignore_index=True,sort=False)
    print(concat_df.shape[0])
    total_df_shape=concat_df.shape[0]
    print('Done concatenating')
    
    ddf = dd.from_pandas(concat_df, npartitions=100)
    
    ddf=ddf.groupby(['fivegram_pos','year'])['count'].sum()
    print('Done grouping')
    ddf=ddf.to_frame().reset_index().compute()
    
    print(ddf.shape[0])
    after_shape=ddf.shape[0]
    
    ddf.to_parquet(
    path=f'{save_path}/df_{row.fcat}.parq', 
    engine='fastparquet',
    compression='snappy',
    row_group_offsets=10_000_000)
    print(f"Finished df {row.fcat} ; Before : {total_df_shape}, After : {after_shape} Change in percentage : {(total_df_shape-after_shape)/total_df_shape*100:0.2f}%")
    print(f'Time taken {time.time()-cur_time} secs')

In [7]:
save_path='/data/dharp/compounds/datasets/entire_df'


In [34]:
pkl_df.groupby('fcat').apply(write_to_parquet)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


""


In [23]:
pkl_df.groupby('fcat')